In [1]:
import os
import tiktoken
import pandas as pd
import numpy as np
import random
import seaborn as sns
from tenacity import retry, stop_after_attempt, wait_fixed
from huggingface_hub import InferenceClient

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(".env")

OPENAI_KEY = os.environ["OPENAI_KEY"]
HF_TOKEN = os.environ['HF_TOKEN']


GPT3 = "gpt-3.5-turbo-1106"
GPT4 = "gpt-4-1106-preview"
ZEPHYR = "meta-llama/Llama-2-7b-chat-hf"


def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens


# @retry(wait=wait_fixed(15), stop=stop_after_attempt(4))
def llm(user_prompt, model, temperature=0.3):
    model_kwargs = {"temperature": temperature}
    user_prompt = user_prompt[:3700]

    client = OpenAI(api_key=OPENAI_KEY)

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": user_prompt}],
        stream=False,
        **model_kwargs,
    )

    output = response.choices[0].message.content

    return output

def hf_llm(text, model, params={'max_new_tokens': 500, 'temperature': 0.3}):

    prompt = f"<s>[INST]{text}[/INST]"
    client = InferenceClient(model=model, token=HF_TOKEN)

    output = client.text_generation(prompt, **params)
    return output

/Users/seansmith/anaconda3/envs/airline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
decomposition_prompt_template = lambda primary_goal, personalization, history: f"""I want to accomplish the main goal of {primary_goal}
To better assist me, please break down the problem into sub-problems.
Each sub-problem should help me to solve the original problem.
Make it so that each sub-problem is not trival and can be helpful.
Take my context and personalizeation cues to personalize the sub-problems.
Make sure each sub-problem is concise and less than 15 words.

Personalization Cue: {personalization}
My Context: {history}

Return a numbered list with a maximum of 8 sub-problems, each on a new line.

Output: """

options_prompt_template = lambda primary_goal, task, personalization, history: f"""
Here is one of the sub-query to help answer the main query.
Go into details to help me with the sub -query.
Show me some options to personalize and choose from.
Be concrete and make sure the options are valid choices to finish the task in sub-query.
When coming up with options, make sure they are diverse and representative of multiple demographics, cultures, and viewpoints.
Output a numbered list of options for me to choose from, with each option on a new line. Provide helpful details.

Primary-query: {primary_goal}
Sub-query: {task}
Personalization Cue: {personalization}
My Context: {history}

Output:"""

summarize_prompt_template = lambda primary_goal, history: f"""I want to solve the problem in the primary-query.
I've loaded all of my research in the history so you can see what I've done so far. 
The history is in the schema {{problem2solve: solutions}}.
Please synthesize my history into a final report about the primary-query.
Make sure to remove any duplicate information if it exists in the history.

Primary-query: {primary_goal}
History: 
{history}

Output:"""

Workflow:
1. User enters a main query, this is passed to the `decomposition` endpoint
2. `decomposition` endpoint returns 8 sub-tasks that are represented as cards
3. When a user clicks on a sub-task, they make a call to the `options` endpoint, this opens a new window
4. The `options` endpoint returns a primary answer, as well as secondary answers that may be helpful
5. A user selects which answers it likes best from the options endpoint which are stored.
6. The user repeats 3-5 until they feel like they have sufficiently explored the sub-tasks
7. The user calls the `summarize` endpoint which produces a final output of all the options they have selected

In [3]:
def decomposition_endpoint(primary_goal, personalization, history):
    history = "None" if len(history) == 0 else "\n".join([f"- {x}" for x in history])
    prompt = decomposition_prompt_template(primary_goal, personalization, history)
    output = llm(prompt, GPT3)
    output = [x.lstrip("0123456789. ") for x in output.split("\n")]
    return output


def options_endpoint(primary_goal, task, personalization, history):
    history = "None" if len(history) == 0 else "\n".join([f"- {x}" for x in history])
    prompt = options_prompt_template(primary_goal, task, personalization, history)
    output = llm(prompt, GPT3)
    output = [x.lstrip("0123456789. ") for x in output.split("\n")]
    return output


def summarize_endpoint(primary_goal, history):
    history = "None" if len(history) == 0 else "\n".join([f"- {x}" for x in history])
    prompt = summarize_prompt_template(primary_goal, history)
    output = llm(prompt, GPT3)
    return output

## Step 1: User selects a primary Goal

In [4]:
primary_goal = 'I want to plan a trip to Tokyo'
personalization = 'I have a walking disability and travel in a wheelchair'
history = []
sub_tasks = decomposition_endpoint(primary_goal, personalization, history)

In [5]:
print('\n'.join(sub_tasks))

Research wheelchair-accessible attractions and activities in Tokyo.
Find wheelchair-accessible accommodations in Tokyo.
Determine the most wheelchair-friendly transportation options in Tokyo.
Plan accessible routes and itineraries for sightseeing in Tokyo.
Check for accessible facilities and amenities in Tokyo, such as restrooms and restaurants.
Research local laws and regulations regarding accessibility for wheelchair users in Tokyo.
Contact local disability organizations or travel agencies for advice and assistance.
Create a backup plan in case of unexpected accessibility challenges during the trip.


## Step 2: User explores some sub-tasks
This code right here simulates the UI

In [6]:
explored_subtasks = random.sample(sub_tasks, 3)
for task in explored_subtasks:
    task_options = options_endpoint(primary_goal, task, personalization, history)
    selected_options = random.sample(task_options, 3)
    history.append({"sub_task": task, "selected_options": selected_options})

In [7]:
print(history)

[{'sub_task': 'Research wheelchair-accessible attractions and activities in Tokyo.', 'selected_options': ['Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation.', 'Visit the Tokyo National Museum, which is wheelchair accessible and offers a wide range of exhibits and artifacts from Japanese history and culture.', "Explore the Senso-ji Temple in Asakusa, which has wheelchair ramps and accessible pathways, and is one of Tokyo's most famous and historic temples."]}, {'sub_task': 'Research local laws and regulations regarding accessibility for wheelchair users in Tokyo.', 'selected_options': ['Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation. This option provides a peaceful and scenic experience for wheelchair users.', "Explore the Senso-ji Temple in Asakusa, which has 

## Step 3: User gets a final summary

In [12]:
print('history:' , history)
print('primary_goal:' , history)

history: [{'sub_task': 'Research wheelchair-accessible attractions and activities in Tokyo.', 'selected_options': ['Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation.', 'Visit the Tokyo National Museum, which is wheelchair accessible and offers a wide range of exhibits and artifacts from Japanese history and culture.', "Explore the Senso-ji Temple in Asakusa, which has wheelchair ramps and accessible pathways, and is one of Tokyo's most famous and historic temples."]}, {'sub_task': 'Research local laws and regulations regarding accessibility for wheelchair users in Tokyo.', 'selected_options': ['Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation. This option provides a peaceful and scenic experience for wheelchair users.', "Explore the Senso-ji Temple in Asakusa, w

In [8]:
final_summary = summarize_endpoint(primary_goal, history)

In [9]:
print(final_summary)

Final Report:

After conducting thorough research, the following solutions have been identified for planning a trip to Tokyo with a focus on wheelchair accessibility:

1. Research wheelchair-accessible attractions and activities in Tokyo:
   - Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation.
   - Visit the Tokyo National Museum, which is wheelchair accessible and offers a wide range of exhibits and artifacts from Japanese history and culture.
   - Explore the Senso-ji Temple in Asakusa, which has wheelchair ramps and accessible pathways, and is one of Tokyo's most famous and historic temples.

2. Research local laws and regulations regarding accessibility for wheelchair users in Tokyo:
   - Take a stroll through the beautiful Shinjuku Gyoen National Garden, which has paved pathways and accessible facilities, perfect for enjoying nature and relaxation. This option provides 

## Some Parting Thoughts


Here's what the schema to basically every endpoint would be: 
```
{
    "primary_goal": "",
    "personalization_cue": "",
    "history": [
        {
            "sub_task": ""
            "selected_options": [""]
        }
    ]
}
```